In [2]:
import os
import re
import json 

import pandas as pd 
import numpy as np 

import matplotlib.pyplot as plt 
from matplotlib import colors
import seaborn as sns 
import plotly.graph_objects as go 
import plotly.express as px 
from plotly.offline import init_notebook_mode
from plotly.subplots import make_subplots

from IPython.display import HTML, display

init_notebook_mode(connected=True)

In [3]:
YEARS = [2018, 2019, 2020]


cities_dir = 'Cities/Cities Responses/'
cc_dir = 'Corporations/Corporations Responses/Climate Change/'
ws_dir = 'Corporations/Corporations Responses/Water Security/'

cities = [pd.read_csv(cities_dir + f'{year}_Full_Cities_Dataset.csv') for year in YEARS]
cc = [pd.read_csv(cc_dir + f'{year}_Full_Climate_Change_Dataset.csv') for year in YEARS]
ws = [pd.read_csv(ws_dir + f'{year}_Full_Water_Security_Dataset.csv') for year in YEARS]

cities_dir_disc = 'Cities/Cities Disclosing/'
cc_dir_disc = 'Corporations/Corporations Disclosing/Climate Change/'
ws_dir_disc = 'Corporations/Corporations Disclosing/Water Security/'

cities_disc = [pd.read_csv(cities_dir_disc + f'{year}_Cities_Disclosing_to_CDP.csv') for year in YEARS]
cc_disc = [pd.read_csv(cc_dir_disc + f'{year}_Corporates_Disclosing_to_CDP_Climate_Change.csv') for year in YEARS]
ws_disc = [pd.read_csv(ws_dir_disc + f'{year}_Corporates_Disclosing_to_CDP_Water_Security.csv') for year in YEARS]

In [4]:
NUM_Q = 'Question Number'
NUM_R = 'Row Number'
NUM_C = 'Column Number'

NAME_Q = 'Question Name'
NAME_R = 'Row Name'
NAME_C = 'Column Name'

ANS = 'Response Answer'

In [5]:
questions = {'cities':{}, 'cc':{}, 'ws':{}}

def qnum_sorter(qnum):
    sections = qnum.split('.')
    if len(sections) == 1: return [16, 0, '']
    first = re.search(r'\d+', sections[0])[0]
    second = re.search(r'\d+', sections[1])[0]
    letter = re.search(r'[a-z]', sections[1])
    return [int(first), int(second), letter[0] if letter else '']

for name, data in zip(('cities', 'cc', 'ws'), (cities, cc, ws)):
    if name == 'cities':
        NUM_Q = 'Question Number'
        ANS = 'Response Answer'
        ORG = 'Organization'
    else:
        NUM_Q = 'question_number'
        ANS = 'response_value'
        ORG = 'organization'
    for year, df in zip(YEARS, data):
        total = len(df[ORG].unique())
        threshold = total * 0.75
        dff = df\
            .dropna(axis=0, subset=[ANS])\
            .groupby(NUM_Q)\
            .agg({ORG: ['nunique']})\
            .droplevel(level=1, axis=1)\
            .sort_values(ORG, ascending=False)\
            .rename(columns={ORG:'Unique Organizations Answered'})
        dff = dff[dff['Unique Organizations Answered'] > threshold]
        questions[name][year] = sorted(dff.index.values.tolist(), key=qnum_sorter)
print(questions)

{'cities': {2018: ['0.1', '0.3', '0.4', '0.5', '0.6', '0.8', '1.0', '1.1', '1.2', '1.4', '2.0', '2.2', '2.2a', '2.3', '3.0', '3.1', '4.0', '5.0', '5.0a', '5.1', '5.2', '6.0', '7.0', '8.0', '8.2', '8.3', '9.0', '9.2', '15.0', '15.3', 'Response Language'], 2019: ['0.1', '0.3', '0.4', '0.5', '0.6', '1.0', '1.1', '2.0', '2.1', '3.0', '3.1', '4.0', '5.0', '5.5', '6.1', '7.0', '8.0', '14.0', '14.3', 'Response Language'], 2020: ['0.1', '0.3', '0.4', '0.5', '0.6', '1.0', '1.0a', '1.1', '1.2', '1.3', '1.4', '1.5', '1.6', '1.7', '2.0', '2.0a', '2.0b', '2.0c', '2.0d', '2.1', '2.1a', '2.1b', '2.1c', '2.2', '2.3', '2.3a', '3.0', '3.1', '3.2', '3.2a', '3.2b', '3.4', '3.5', '3.6', '4.0', '4.1', '4.2', '4.3', '4.3a', '4.4', '4.5', '4.6a', '4.6b', '4.6c', '4.6d', '4.6e', '4.6f', '4.7', '4.8', '4.9', '4.11', '4.12', '4.12a', '4.12b', '4.14', '4.14a', '5.0', '5.0a', '5.0b', '5.0c', '5.0d', '5.0e', '5.1', '5.2', '5.2a', '5.3', '5.3a', '5.5', '5.5a', '5.5b', '6.0', '6.1', '6.2', '6.2a', '6.3', '6.4', '6.5'

In [29]:
cc[2].columns

Index(['account_number', 'organization', 'survey_year',
       'response_received_date', 'accounting_period_to', 'ors_response_id',
       'submission_date', 'page_name', 'module_name', 'question_number',
       'question_unique_reference', 'column_number', 'column_name',
       'table_columns_unique_reference', 'row_number', 'row_name',
       'data_point_name', 'data_point_id', 'response_value', 'comments'],
      dtype='object')

In [30]:
from plotly.offline import init_notebook_mode
from plotly.subplots import make_subplots
import plotly.graph_objects as go 
init_notebook_mode(connected=True)
rows = ('cities', 'cc', 'ws')

fig = make_subplots(rows=3, cols=3, shared_yaxes='all', column_titles=YEARS, row_titles=rows, x_title='Sections in Most Responded Questions (HOVER)', y_title='Frequency in Most Responded Questions')
sections = {}
for df, (name, data) in zip((cities, cc, ws), questions.items()):
    if name == 'cities':
        NUM_Q = 'Question Number'
        ANS = 'Response Answer'
        SECT = 'Parent Section'
        ORG = 'Organization'
    else:
        NUM_Q = 'question_number'
        ANS = 'response_value'
        SECT = 'module_name'
        ORG = 'organization'
    sections[name] = {}
    for i, (year, qs) in enumerate(data.items()):
        dff = df[i][df[i][NUM_Q].isin(qs)].groupby([NUM_Q]).agg('first')[SECT].value_counts()
        fig.add_trace(go.Bar(
            x=dff.index, y=dff.values, showlegend=False
        ), row=rows.index(name)+1, col=i+1)
fig.update_xaxes(showticklabels=False)
fig.show()

In [46]:
pd.set_option('display.max_columns', 30)
for dfs, (name, data) in zip((cities, cc, ws), questions.items()):
    if name == 'cities':
        NUM_Q = 'Question Number'
        ANS = 'Response Answer'
        SECT = 'Parent Section'
        ORG = 'Organization'
        NUM_C = 'Column Number'
        NUM_R = 'Row Number'
    else:
        NUM_Q = 'question_number'
        ANS = 'response_value'
        SECT = 'module_name'
        ORG = 'organization'
        NUM_C = 'column_number'
        NUM_R = 'row_number'
    for year, df in zip(YEARS, dfs):
        print(name.upper(), year)
        dff = df\
            .groupby([NUM_Q, NUM_C])\
            .agg([
                lambda s: int(s.astype('str').str.len().mean()), 
                lambda s: int(s.astype('str').str.len().median()),
                'count'])\
            [ANS]\
                .rename(columns={'<lambda_0>':'Mean Length', '<lambda_1>':'Median Length'})\
                .sort_values(['Median Length', 'Mean Length', 'count'], ascending=False)\
                .head(25)\
                .T
        display(dff)

CITIES 2018


Question Number,0.1,1.0,1.1,2.0a,2.0c,5.1a,1.2,1.4a,2.3,8.2b,8.4,5.0a,11.6a,4.0a,2.2a,3.1b,3.3,2.2b,8.1,5.2,2.4,3.6,2.1a,3.5,9.3a
Column Number,2,0,0,2,1,2,2,2,2,1,5,2,2,3,10,2,4,2,5,4,3,3,4,2,0
Mean Length,1217,1000,849,518,610,365,479,426,371,489,347,319,378,211,266,238,275,271,255,344,230,225,217,325,273
Median Length,922,620,517,316,271,240,239,237,222,192,185,184,152,151,149,147,145,141,137,136,133,132,132,121,119
count,467,422,394,265,281,465,330,664,356,486,1436,1231,12,781,1598,90,1185,10,1004,836,923,214,77,108,74


CITIES 2019


Question Number,0.1,2.0a,5.4,1.0a,5.2a,3.0,6.1a,1.1b,5.1,2.1,6.0,8.0a,10.7a,2.2,14.4,5.3a,6.2,8.6a,5.0d,14.3a,13.6,4.11b,1.1a,1.11,8.0b
Column Number,2,2,10,2,0,6,2,2,0,12,2,10,2,3,4,4,5,9,12,4,2,2,3,2,2
Mean Length,1307,600,508,462,392,440,372,223,364,334,304,351,332,241,279,187,351,304,605,200,230,201,293,98,216
Median Length,969,330,306,281,270,250,239,214,184,184,181,161,160,155,142,138,137,135,128,126,124,122,115,113,107
count,792,497,3139,1807,190,2599,904,4,135,2982,2002,303,62,2233,1016,32,1180,172,73,984,49,171,933,95,114


CITIES 2020


...

question_number W1.4a W-EU3.1a W-FB3.1a W1.4a W4.3a W-OG3.1a W-MM3.2a W4.2  \
column_number    4.0      4.0      5.0   3.0   3.0      5.0      3.0  13.0   
Mean Length       612      830      801   593   697      608      830  540   
Median Length     716      687      651   611   605      525      453  452   
count              87       49       40   106   313       27       35  327   

question_number W1.4b W1.1 W8.1 W1.4b W9.1a      W8.1b W-MM3.2a W4.2a W4.2  \
column_number    6.0  3.0  3.0   5.0   3.0  4.0   8.0      2.0   14.0 6.0    
Mean Length       444  496  618   430   414  477   441      342   526  445   
Median Length     441  429  410   381   380  377   358      351   346  346   
count             111  387  170   117   220  212   222       35    88  340   

question_number W8.1b W4.2a W1.2j W-FB3.1a W3.3b W1.2b W3.3c  
column_number    4.0   7.0   3.0      3.0   2.0   3.0   2.0   
Mean Length       442   438   426      474   420   338   383  
Median Length     345   345   332      316   284   258   237  
count             229    89   124       40  1139   519  1695

WS 2019


question_number W4.3a W1.1 W4.2 W8.1 W4.2 W9.1a      W1.2b W8.1a W8.1b W5.1  \
column_number    3.0  3.0  6.0  3.0  16.0  3.0  4.0   3.0   11.0  4.0  15.0   
Mean Length       567  518  336  507  375   288  342   286   213   330  178   
Median Length     473  365  250  249  249   191  159   137   113   109  109   
count             312  477  342  191  330   235  230   578   227   217  637   

question_number W0.6a W8.1b W3.3b W3.3c W8.1a W4.3a  W1.2 W4.2 W5.1c W4.2  \
column_number    2.0   8.0   2.0   2.0   5.0   10.0  2.0  18.0  5.0  14.0   
Mean Length       181   320   329   296   202   217   204  148   131  164   
Median Length     104   102    97    97    89    85    84   74    69   64   
count             194   211  1264  1888   236   294  2290  323   579  318   

question_number W1.2h W8.1      W8.1a  
column_number    4.0  2.0  1.0   6.0   
Mean Length       165   50   67    32  
Median Length      54   44   43    38  
count             959  205  262   235

WS 2020


question_number W4.3a W1.1 W8.1 W4.2      W1.2b W3.3c W3.3b W8.1a W4.3a W8.1b  \
column_number    3.0  3.0  3.0  4.0  14.0  3.0   2.0   2.0   11.0  10.0  4.0    
Mean Length       703  610  608  397  416   339   352   405   244   249   359   
Median Length     579  455  348  249  217   191   147   144   143   116   113   
count             353  532  217  356  348   647  2286  1518   285   340   232   

question_number      W5.1 W8.1a W1.4b  W1.2 W0.6a W4.2      W3.3a W1.2h W8.1  \
column_number   8.0  25.0  5.0   5.0   2.0   2.0  16.0 12.0  7.0   4.0  2.0    
Mean Length      357  223   220   324   219   197  201  234   284   174   53   
Median Length    105  105   100    98    93    90   84   72    62    60   44   
count            229  651   292   175  2324   204  333  323   458  1030  233   

question_number      W8.1a W8.1b  
column_number   1.0   6.0   1.0   
Mean Length       74    38    39  
Median Length     43    38    36  
count            290   290   234

## Random extra code for Raphael looking at factors affecting response to climate change hazards and the percent of countries that found this to be challenging

In [53]:
df = cities[2]
NUM_Q = 'Question Number'
ANS = 'Response Answer'
NUM_C = 'Column Number'
ORG = 'Organization'
NUM_R = 'Row Number'
dff = df[df[NUM_Q] == '2.2']
dfff = dff[dff[NUM_C] == 1]
dffc = dff[dff[NUM_C] == 2]
dfff['Challenge'] = dfff.apply(lambda row: dffc[(dffc[ORG] == row[ORG]) & (dffc[NUM_R] == row[NUM_R])][ANS].values[0], axis=1)

In [59]:
pd.set_option('display.max_rows', 200)
data = dfff.groupby(ANS).agg({'Challenge': lambda s: round(100 * len(s[s == 'Challenges'].index) / len(s.index), 2)}).sort_values('Challenge', ascending=False).rename_axis('Factors')
data[~data.index.str.contains('Other, please specify')]

,Challenge
Factors,
Safety and security: Disaster clean up,100.00
Underemployment,100.00
Inequality,96.47
Poverty,93.58
Rapid urbanization,92.22
Migration,89.74
Unemployment,89.47
Cost of living,87.36
Housing,86.61
